In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_curve, auc


: 

In [ ]:

class MultiModalDataset(Dataset):
    def __init__(self, audio_dir, visual_dir, split_file, annotation_file):
        self.audio_dir = audio_dir
        self.visual_dir = visual_dir

        # Load file IDs from split file
        with open(split_file, 'r') as f:
            fids = f.read().splitlines()

        # Generate paths to audio and visual files
        self.audio_files = [os.path.join(audio_dir, f"{i}.npy") for i in fids]
        self.visual_files = [os.path.join(visual_dir, f"{i}.npy") for i in fids]

        # Generate labels from file IDs
        self.labels = [1 if 'lie' in fid.lower() else 0 for fid in fids]

        # Load annotations
        self.annotations = pd.read_csv(annotation_file, index_col=0)
        missing_fids = [fid.split('/')[-1] for fid in fids if fid not in self.annotations.index]
        if missing_fids:
            raise ValueError(f"Missing file ids in annotation file: {missing_fids}")
        self.annotations = self.annotations.loc[fids].values

        # Ensure that files exist
        for i in self.audio_files:
            if not os.path.exists(i):
                raise FileNotFoundError(f"File not found: {i}")
        for i in self.visual_files:
            if not os.path.exists(i):
                raise FileNotFoundError(f"File not found: {i}")
    
    def __getitem__(self, idx):
        # Load audio and visual features
        audio_features = np.load(self.audio_files[idx])
        visual_features = np.load(self.visual_files[idx])
        
        # Ensure the same length for both modalities
        min_length = min(audio_features.shape[0], visual_features.shape[0])
        audio_features = audio_features[:min_length]
        visual_features = visual_features[:min_length]

        # Get annotations for this sample
        annotation = self.annotations[idx]
        annotation_repeated = np.tile(annotation, (min_length, 1))

        # Concatenate audio, visual features, and annotations along the last dimension
        concat_data = audio_features
        #np.concatenate([audio_features, visual_features], axis=-1)
        
        # Convert to tensor
        concat_data = torch.tensor(concat_data, dtype=torch.float32)
        
        # Get the label for this sample
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long)

        return concat_data, label
        
    def __len__(self):
            return len(self.audio_files)
    
    


def collate_fn(batch):
    data, labels = zip(*batch)
    
    # Determine the max length of the sequences
    max_len = max([i.shape[0] for i in data])
    
    # Pad sequences to the max length
    padded_batch = [torch.nn.functional.pad(i, (0, 0, 0, max_len - i.shape[0])) for i in data]
    
    # Stack into a tensor
    data = torch.stack(padded_batch)
    
    # Stack labels into a tensor
    labels = torch.tensor(labels)
    
    return data, labels

# Example usage
audio_dir = "/data/lie_detection/clips_umich/opensmile_extraction"
visual_dir = "/data/lie_detection/clips_umich/viT_50hz"
split_file = "/data/lie_detection/clips_umich/train.txt"
annotation_file = "/home/blastaistudent/proj-lie-detection/annotation_id_as_index.csv"

dataset = MultiModalDataset(audio_dir, visual_dir, split_file, annotation_file)
print(f"dataset sample shape: {dataset[0][0].shape}, label: {dataset[0][1]}")

dataloader = DataLoader(dataset, batch_size=12, shuffle=True, num_workers=2, collate_fn=collate_fn)

batch = next(iter(dataloader))
print(batch[0].shape, batch[1].shape)


dataset sample shape: torch.Size([1, 6885]), label: 1


/home/blastaistudent/miniconda3/envs/ld/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


torch.Size([12, 1, 6885]) torch.Size([12])


In [36]:
dataset[0][0][0]

tensor([ 2.5671,  0.3718,  0.0000,  ...,  0.0892,  0.2583, -0.0228])

In [27]:
batch_size=12
val_split="/data/lie_detection/clips_umich/val.txt"
valset = MultiModalDataset(audio_dir, visual_dir, val_split,annotation_file)
valloader = DataLoader(valset, batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn)

/home/blastaistudent/miniconda3/envs/ld/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [28]:

test_split = "/data/lie_detection/clips_umich/test.txt"
testset = MultiModalDataset(audio_dir, visual_dir, test_split, annotation_file)
testloader = DataLoader(testset, batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn)

/home/blastaistudent/miniconda3/envs/ld/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [29]:
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader


X =[]
y =  dataloader.dataset.labels

# Concatenate all the features and labels into single tensors
for i in range(len(dataloader.dataset)):
    X.append(dataloader.dataset[i][0])

print(len(X))
print(len(y))


X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)

84
84


In [11]:
X = np.array(X)

In [30]:








def numpy_to_data(X, y, k=5):
    data_list = []
    for i in range(len(X)):
        x = torch.tensor(X[i], dtype=torch.float)
        
        k_neighbors = 1
        
        # Generate k-nearest neighbor graph
        adj_matrix = kneighbors_graph(X[i], k_neighbors, mode='connectivity', include_self=True)
        edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
        
        label = torch.tensor(y[i], dtype=torch.long)
        data = Data(x=x, edge_index=edge_index, y=label.unsqueeze(0))
        data_list.append(data)
    return data_list

train_data = numpy_to_data(X_train, y_train, k=5)
test_data = numpy_to_data(X_test, y_test, k=5)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)


/tmp/ipykernel_1140490/2086615131.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(X[i], dtype=torch.float)
/tmp/ipykernel_1140490/2086615131.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(X[i], dtype=torch.float)
/tmp/ipykernel_1140490/2086615131.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(X[i], dtype=torch.float)
/tmp/ipykernel_1140490/2086615131.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import torch_geometric.nn as pyg_nn

class ComplexGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, k_neighbors, dropout=0.5):
        super(ComplexGCN, self).__init__()
        self.k = k_neighbors
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.fc1 = nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)
        self.dropout = dropout
        self.batch_norm1 = nn.BatchNorm1d(hidden_channels)
        self.batch_norm2 = nn.BatchNorm1d(hidden_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.tanh(x)
        x = self.batch_norm1(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.batch_norm2(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)
        
        x = global_mean_pool(x, data.batch)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        
        return x


In [58]:
import torch.optim as optim


In [67]:
    


in_channels = 6885  # Number of input features per node
hidden_channels = 64
out_channels = 1  # Binary classification
k_neighbors = 5

model = ComplexGCN(in_channels, hidden_channels, out_channels, k_neighbors)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

def train(model, loader):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.squeeze(), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

def evaluate(model, loader):
    model.eval()
    correct = 0
    y_true = []
    y_pred = []
    
    for data in loader:
        out = model(data)
        pred = out.squeeze().round()
        y_true.extend(data.y.cpu().detach().numpy())
        y_pred.extend(pred.cpu().detach().numpy())
        correct += pred.eq(data.y.float()).sum().item()
    
    return correct / len(loader.dataset), y_true, y_pred

num_epochs = 40
for epoch in range(num_epochs):
    train_loss = train(model, train_loader)
    test_acc, y_true, y_pred = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# After training loop, generate the classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=0))


Epoch 1/40, Loss: 50.0000, Test Accuracy: 0.5385
Epoch 2/40, Loss: 53.4483, Test Accuracy: 0.5385
Epoch 3/40, Loss: 48.2759, Test Accuracy: 0.5385
Epoch 4/40, Loss: 48.2759, Test Accuracy: 0.5385
Epoch 5/40, Loss: 51.7241, Test Accuracy: 0.5385
Epoch 6/40, Loss: 55.1724, Test Accuracy: 0.5385
Epoch 7/40, Loss: 56.8966, Test Accuracy: 0.5385
Epoch 8/40, Loss: 50.0000, Test Accuracy: 0.5385
Epoch 9/40, Loss: 53.4483, Test Accuracy: 0.5385
Epoch 10/40, Loss: 50.0000, Test Accuracy: 0.5385
Epoch 11/40, Loss: 51.7241, Test Accuracy: 0.5385
Epoch 12/40, Loss: 51.7241, Test Accuracy: 0.5385
Epoch 13/40, Loss: 50.0000, Test Accuracy: 0.5385
Epoch 14/40, Loss: 55.1724, Test Accuracy: 0.5385
Epoch 15/40, Loss: 51.7241, Test Accuracy: 0.5385
Epoch 16/40, Loss: 46.5517, Test Accuracy: 0.5385
Epoch 17/40, Loss: 48.2759, Test Accuracy: 0.5385
Epoch 18/40, Loss: 51.7241, Test Accuracy: 0.5385
Epoch 19/40, Loss: 48.2759, Test Accuracy: 0.5385
Epoch 20/40, Loss: 44.8276, Test Accuracy: 0.5385
Epoch 21/